In [1]:
import pandas as pd

location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.New_colias/61.New_alignment_colias"

diapause = pd.read_csv(f"{location}/2.ERC_plot/ERC_correlation_diapause_longform.csv")

# genes = ["8751at7088","period","3165at7088","3811at7088","4043at7088","13263at7088","12510at7088","2560at7088","8809at7088","1846at7088","5474at7088","5423at7088","3362at7088","1709at7088","2036at7088","5326at7088","10001at7088","4842at7088","6081at7088","1390at7088","cycle","8044at7088","16035at7088","12354at7088","12730at7088","1422at7088","6031at7088","5679at7088"]

# genes = ["8751at7088","period","3165at7088","3811at7088","4043at7088","8809at7088","12510at7088","13263at7088","1846at7088","5423at7088","3362at7088","1709at7088","5474at7088","2560at7088","2036at7088","5326at7088","cycle","10001at7088","1390at7088","8044at7088","6081at7088","16035at7088","4842at7088","5679at7088","1422at7088"]

#just Cluster
# genes = ['3811at7088', '5679at7088', '8044at7088', '5423at7088', '12510at7088', '6081at7088', 'period', '1422at7088', '1846at7088', '2036at7088', 'cycle', '5474at7088', '3362at7088', '1709at7088']


#full genes not just cluster






In [5]:
genes = ["8809at7088","5423at7088","4043at7088","3362at7088","2036at7088","3811at7088","2560at7088","cycle","5474at7088","16035at7088","1709at7088","8044at7088","10001at7088","8751at7088","6081at7088","3165at7088","4842at7088","period","12510at7088","1422at7088","5679at7088","5326at7088","1390at7088","13263at7088","1846at7088"]

anno_names_list = []

location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/unTrimmed/3.For_tree"
gene_anno_dic = {}
with open(f"/mnt/g/My Drive/Circadian Rhythm Genes Project/20.RERconverge/2.Modified_diapause/correlation_with_annotation.csv", 'r') as cor_file:
    for lines in cor_file.readlines()[1:]:
        gene_name = lines.split(",")[0].split("_")[0]
        # print(gene_name)
        anno_name = lines.split(",")[-1].replace('"', '')
        if len(anno_name) < 2:
            # print(gene_name,anno_name)
            anno_name = gene_name
        
        gene_anno_dic[gene_name]= anno_name.strip()




with open(f"/mnt/g/My Drive/Circadian Rhythm Genes Project/63.BUSCO_annotation/busco_names.csv", 'r') as cor_file:
    for lines in cor_file.readlines()[1:]:
        lines_split = lines.strip().split(",")
        # print(lines_split)
        gene_name = lines_split[0]
        anno_name = lines_split[1]
        if len(anno_name) > 7:
            
            anno_name = anno_name[:7]
            
            if anno_name[:7] not in anno_names_list:
                anno_name = anno_name[:7]
                anno_names_list.append(anno_name)
            else:
                counter = 0
                while True:
                    if anno_name[:7][:-2] + "_" + str(counter) not in anno_names_list:
                        anno_name = anno_name[:7][:-2] + "_" + str(counter)
                        anno_names_list.append(anno_name)
                        break
                    else:
                        counter += 1
                    # if anoo
        
            
        if gene_name not in gene_anno_dic:
            
            gene_anno_dic[gene_name]= anno_name
            # print(gene_name)
            continue
        if gene_anno_dic[gene_name] == gene_name:
            gene_anno_dic[gene_name]= anno_name
            # print(gene_name)
        if gene_name in gene_anno_dic:
            if len(gene_anno_dic[gene_name]) > 7:
                if gene_anno_dic[gene_name] [:7] not in anno_names_list:
                    # anno_name = anno_name[:7]
                    anno_names_list.append(gene_anno_dic[gene_name] [:7])
                    gene_anno_dic[gene_name] = gene_anno_dic[gene_name] [:7]
                else:
                    counter = 0
                    while True:
                        if (gene_anno_dic[gene_name] [:7][:-2] + "_" + str(counter)) not in anno_names_list:
                            # anno_name = anno_name[:-2] + "_" + str(counter)
                            anno_names_list.append(gene_anno_dic[gene_name] [:7][:-2] + "_" + str(counter))
                            gene_anno_dic[gene_name] =gene_anno_dic[gene_name] [:7][:-2] + "_" + str(counter)
                            break
                        else:
                            counter += 1

In [6]:
filtered_df = diapause[(diapause['Gene1'].isin(genes)) & (diapause['value']<1) &(diapause['Gene2'].isin(genes))]
# filtered_df = diapause[(diapause['Gene1'].isin(genes)) & (diapause['value']>0.4)]


filtered_df = filtered_df.rename(columns={"value": "Correlation"})

def check_annotation(name):
    return gene_anno_dic.get(name, name)

filtered_df["Annotation_gene1"] = filtered_df["Gene1"].apply(check_annotation)
filtered_df["Annotation_gene2"] = filtered_df["Gene2"].apply(check_annotation)


# filtered_df = filtered_df.drop('Unnamed: 0', axis=1)
filtered_df["Correlation"].fillna(1, inplace=True)
# print(filtered_df)

In [7]:
cluster_dict = {}
cluster_dict[0] = []

for row in filtered_df.itertuples():
    if row.Correlation > 0.4:
        presence = 0
        # print(cluster_dict)
        for key in cluster_dict:
            # print(cluster_dict[key])
            if row.Gene1 in cluster_dict[key] and row.Gene2 not in cluster_dict[key]:
                cluster_dict[key].append(row.Gene2)
                presence = 1
            if row.Gene2 in cluster_dict[key] and row.Gene1 not in cluster_dict[key]:
                cluster_dict[key].append(row.Gene1)
                presence = 1
            if row.Gene1 in cluster_dict[key] or row.Gene2 in cluster_dict[key]:
                presence = 1
        if presence == 0:
            # print(cluster_dict, row.Gene1, row.Gene2)
            cluster_dict[key+1] = [row.Gene1, row.Gene2]
                
                
                
                
            
    # print(row.Correlation)
for key,value in cluster_dict.items():
    print(key, value, len(value))

0 [] 0
1 ['8809at7088', '13263at7088'] 2
2 ['5423at7088', '3362at7088', '2036at7088', '1846at7088', '3811at7088', '1709at7088', '12510at7088', '5679at7088', 'cycle', 'period', '5474at7088', '8044at7088', '6081at7088', '1422at7088'] 14


In [8]:
genes = cluster_dict[2]
# filtered_df = diapause[(diapause['Gene1'].isin(genes)) & (diapause['value']<1) &(diapause['Gene2'].isin(genes))]
filtered_df = diapause[(diapause['Gene1'].isin(genes)) & (diapause['value']>0.4)& (diapause['value']<1)]


filtered_df = filtered_df.rename(columns={"value": "Correlation"})

def check_annotation(name):
    return gene_anno_dic.get(name, name)

filtered_df["Annotation_gene1"] = filtered_df["Gene1"].apply(check_annotation)
filtered_df["Annotation_gene2"] = filtered_df["Gene2"].apply(check_annotation)

filtered_df['edge_key'] = filtered_df.apply(lambda row: tuple(sorted([row['Gene1'], row['Gene2']])), axis=1)
filtered_df = filtered_df.drop_duplicates('edge_key')



# filtered_df = filtered_df.drop('Unnamed: 0', axis=1)
filtered_df["Correlation"].fillna(1, inplace=True)
# print(filtered_df)

In [9]:
location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.New_colias/61.New_alignment_colias/12.Rho_networks/"
# filtered_df.to_csv(f"{location}/Filtered_set_core_set.csv", index = False)


unique_species = filtered_df['Gene1'].unique().tolist()



print((unique_species), len(unique_species))

unique_species+= (filtered_df['Gene2'].unique().tolist())
print(set(unique_species), len(set(unique_species)), len((unique_species)))
output_rho_set = 'BUSCO,GENE'
with open(f"{location}/rho_set_core_25.txt", "w") as out_file:
    for name_busco in set(unique_species):
        output_rho_set += f"\n{name_busco},{gene_anno_dic[name_busco]}"
    out_file.write(output_rho_set)

['5423at7088', '3362at7088', '2036at7088', '3811at7088', 'cycle', '5474at7088', '1709at7088', '8044at7088', '6081at7088', 'period', '12510at7088', '1422at7088', '5679at7088', '1846at7088'] 14
{'12123at7088', '7681at7088', '12075at7088', '1837at7088', '5091at7088', '3166at7088', '8065at7088', '10437at7088', '5884at7088', '11592at7088', '5679at7088', '9535at7088', '11288at7088', '2885at7088', '13356at7088', '2166at7088', '7855at7088', '721at7088', '2906at7088', '10387at7088', '5821at7088', '3126at7088', '5650at7088', '2829at7088', '10186at7088', '1330at7088', '10101at7088', '5722at7088', '6607at7088', '4657at7088', '7993at7088', '3026at7088', '4416at7088', '11275at7088', '9682at7088', '652at7088', '2234at7088', '6835at7088', '8100at7088', '1220at7088', '2905at7088', '13447at7088', '5563at7088', '15565at7088', '998at7088', '13157at7088', '10896at7088', '1709at7088', '10142at7088', '741at7088', '5858at7088', '5508at7088', '9603at7088', '1685at7088', '5199at7088', '11416at7088', '7415at7088

In [11]:
genes = set(unique_species)
# filtered_df = diapause[(diapause['Gene1'].isin(genes)) & (diapause['value']<1) &(diapause['Gene2'].isin(genes))]
# filtered_df = diapause[(diapause['Gene1'].isin(genes)) & (diapause['value']>0.4)& (diapause['value']<1) &(diapause['Gene2'].isin(genes))]
filtered_df = diapause[(diapause['Gene1'].isin(genes))& (diapause['value']<1) &(diapause['Gene2'].isin(genes))]


filtered_df = filtered_df.rename(columns={"value": "Correlation"})

def check_annotation(name):
    return gene_anno_dic.get(name, name)

filtered_df["Annotation_gene1"] = filtered_df["Gene1"].apply(check_annotation)
filtered_df["Annotation_gene2"] = filtered_df["Gene2"].apply(check_annotation)

filtered_df['Annotation_gene1'] = filtered_df['Annotation_gene1'].str.replace('at7088', '', regex=False)
filtered_df['Annotation_gene2'] = filtered_df['Annotation_gene2'].str.replace('at7088', '', regex=False)
filtered_df['Annotation_gene1'] = filtered_df['Annotation_gene1'].str.replace('golgin subfamily A member 6-like', 'golginA', regex=False)
filtered_df['Annotation_gene2'] = filtered_df['Annotation_gene2'].str.replace('golgin subfamily A member 6-like', 'golginA', regex=False)

filtered_df['edge_key'] = filtered_df.apply(lambda row: tuple(sorted([row['Annotation_gene1'], row['Annotation_gene2']])), axis=1)
filtered_df = filtered_df.drop_duplicates('edge_key')



# filtered_df = filtered_df.drop('Unnamed: 0', axis=1)
filtered_df["Correlation"].fillna(1, inplace=True)
print(filtered_df)

filtered_df.to_csv(f"{location}/Filtered_set_core_set.csv", index = False)

                Gene1        Gene2  Correlation Annotation_gene1  \
13725      5969at7088   6051at7088     0.115214            DCAF7   
13729      5969at7088   2979at7088     0.306482            DCAF7   
13731      5969at7088   3994at7088     0.241701            DCAF7   
13740      5969at7088    652at7088     0.387766            DCAF7   
13743      5969at7088   1943at7088     0.519634            DCAF7   
...               ...          ...          ...              ...   
20715003    515at7088  14326at7088     0.391085          DENND5B   
20715016    515at7088   9362at7088     0.018420          DENND5B   
20783418  14318at7088  14326at7088    -0.128283        LOC10_122   
20783431  14318at7088   9362at7088     0.213239        LOC10_122   
20797143  14326at7088   9362at7088     0.089071            RBM38   

         Annotation_gene2              edge_key  
13725               HTR1A        (DCAF7, HTR1A)  
13729             SLCO4A1      (DCAF7, SLCO4A1)  
13731               BTBD1        